## Microscopy Data Processing ##

Use locations of regions of interest, specified in a spreadsheet, to cut out relevant sections of the larger mosaic images. For each specified region:
 - Add a 25 pixel border all the way around
 - Add an additional (w,x,y,z) border where w - z are uniform random numbers between 0 and 25
 - Save the resulting area as slide_roi.tif
 - Save the resulting roi locations (top left coordinate, bottom right coordinate) within the original slide to a csv file

In [6]:
import numpy as np
import matplotlib.pyplot as plt #requires Pillow to read tiffs

In [7]:
data_dir = '../Microscopie optica/'
roi_file = data_dir + 'roi_details.csv'
image_file = data_dir + '%d.tif'
out_file = 'rois/%d_%d.tif'
out_roi_file = 'rois/roi_locations.csv'

In [8]:
roi_data = np.genfromtxt(roi_file, delimiter=',', dtype=None, encoding="utf8")
roi_data = roi_data[1:].astype(int) #drop the header row and convert to integers

Loop through rows of csv file, load the mosaic image (if it's not the same as the previous line), then create the cutout defined by the top left and bottom right coordinates, plus padding.

In [9]:
slide_id = None
allpad = 25
rndpad = (0, 25)
roi_posns=[]

for row in roi_data.astype(int):
    if row[0] != slide_id:
        slide_id = row[0]
        mosaic = plt.imread(image_file%(slide_id,))
    
    roi_id = row[1]
    (w, x, y, z) = [allpad,]*4 + np.random.randint(*rndpad,4)
    paddedgt = mosaic[(row[3]-w):(row[5]+1+x), (row[2]-y):(row[4]+1+z), :]
    roi_posns.append([slide_id, roi_id, (row[2]-y), (row[3]-w), (row[4]+1+z), (row[5]+1+x)]) #save positions in x,y order as above instead of y, x order per numpy
    plt.imsave(out_file%(slide_id,roi_id), paddedgt)

np.savetxt(out_roi_file, roi_posns, delimiter=',')